In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
cookies = {
    'espn_s2': 'AEAt9hEGT8bZRnrplUkxPkbUvRMddvL0GstzcHzyTCkbyY3n4I%2FKzUFStdp5cJA6mvyfPMDWkHdKwE9qs7GVA0vaFh0Qavvfv1TkiAvSNbgl9lUQk0rSUOQJglY77ia2HnS4rjevEfu%2B5KuLnozcWL7gme4wQP2%2BcEMOCiynKAf9gMyV0TtvLeN5bqW70LqZtLuieqVZexxmmAb6IzXibA6u4veHaRKIp9xFkIeuCtjFl2Y0dqOQwwJZp9ErideNV5l351zCohq791jMTwd33oi68rWszlawckLdLo91nKmwPQ%3D%3D',
    'swid': '{D268F2AB-4B60-4E6B-ADDD-B9275BCB1B1B}'
}

In [ ]:
OWNERS = {
    '7': "Dubbe",
    '8': "Ott",
    '9': "Flemming",
    '10': "Bunce",
    '11': "Dykstra", 
    '12': "Cavanagh",
    '13': "Johnson",
    '14': "Harris", 
    '15': "Karos", 
    '16': "Velner", 
    '17': "Flaherty",
    '18': "Quello",
    '19': 'Hubbell',
    '20': 'Demuth'
}

In [ ]:
POSITION_CODES = {
        '1' : 'QB',
        '2' : 'RB',
        '3' : 'WR',
        '4' : 'TE',
        '16' : 'D/ST',
        '5' : 'K'
    }

In [ ]:
STAT_KEYS = {

        '0' : 'pass_attempts',
        '1' : 'completions',
        '3' : 'pass_yards',
        '4' : 'pass_tds',
        '20' : 'interceptions',
        '23' : 'rush_attempts',
        '24' : 'rush_yards',
        '25' : 'rush_tds',
        '41' : 'receptions',
        '42' : 'receiving_yards',
        '43' : 'receiving_tds'
    }

In [ ]:
league_id = 930603
season = 2022
week = 17
league_endpoint = f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?view=mMatchup&view=mMatchupScore&scoringPeriodId={week}'

In [ ]:
r = requests.get(league_endpoint, cookies=cookies).json()
r.keys()

In [ ]:
r['teams']

In [320]:
''' 
Function Description...
'''

def get_weekly_player_stats(leagueID, season, week):

    league_endpoint = f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?view=mMatchup&view=mMatchupScore&scoringPeriodId={week}'

    json = requests.get(league_endpoint, cookies=cookies).json()

    teams = json['teams']

    output = []

    keys = []

   
    for i in range(len(teams)):

        team = teams[i]
        team_id = team['id']

        owner = OWNERS[str(team_id)]

        players = team['roster']['entries']
    

        for player in players:

            name = player['playerPoolEntry']['player']['fullName']
            player_id = player['playerId']
            slot = player['lineupSlotId']
            position_id = player['playerPoolEntry']['player']['defaultPositionId']
            position = POSITION_CODES[str(position_id)]
            starter = 'TRUE' if (slot < 9 or slot == 16 or slot == 17) else 'FALSE'
            is_injured = player['playerPoolEntry']['player']['injured']
            injury_status = player['playerPoolEntry']['player'].get('injuryStatus', None)

            stats = player['playerPoolEntry']['player']['stats']

           

            filtered = [x for x in stats if x['scoringPeriodId'] == week]



            actual_stat = [x for x in filtered if x['statSourceId'] == 0]
            print(len(actual_stat))

            actual_fantasy_points = actual_stat[0]['appliedTotal']
            actual_stats = actual_stat[0]['stats']
            
            proj_stat = [x for x in filtered if x['statSourceId'] == 1]
            proj_fantasy_points = round(proj_stat[0]['appliedTotal'], 1)
            proj_stats = proj_stat[0]['stats']

            
            # PROJECTED STATS

            # QB STATS
            p_pass_attempts = round(proj_stats.get('0', 0), 1)
            p_completions = round(proj_stats.get('1', 0), 1)
            p_pass_yards = round(proj_stats.get('3', 0), 1)
            p_pass_tds = round(proj_stats.get('4', 0), 1)
            p_ints = round(proj_stats.get('20', 0), 1)

            # RB STATS
            p_rush_attempts = round(proj_stats.get('23', 0), 1)
            p_rush_yards = round(proj_stats.get('24', 0), 1)
            p_rush_tds = round(proj_stats.get('25', 0), 1)

            # WR STATS
            p_receptions = round(proj_stats.get('41', 0), 1)
            p_receiving_yards = round(proj_stats.get('42', 0), 1)
            p_receiving_tds = round(proj_stats.get('43', 0), 1)

            #----------------------------------------------------------------------

            # ACTUAL STATS

            # QB STATS
            pass_attempts = round(actual_stats.get('0', 0), 1)
            completions = round(actual_stats.get('1', 0), 1)
            pass_yards = round(actual_stats.get('3', 0), 1)
            pass_tds = round(actual_stats.get('4', 0), 1)
            ints = round(actual_stats.get('20', 0), 1)

            # RB STATS
            rush_attempts = round(actual_stats.get('23', 0), 1)
            rush_yards = round(actual_stats.get('24', 0), 1)
            rush_tds = round(actual_stats.get('25', 0), 1)

            # WR STATS
            receptions = round(actual_stats.get('41', 0), 1)
            receiving_yards = round(actual_stats.get('42', 0), 1)
            receiving_tds = round(actual_stats.get('43', 0), 1)
        
            output.append([season, week, owner, name, player_id, position, starter, is_injured, injury_status, actual_fantasy_points, proj_fantasy_points, p_pass_attempts, p_completions, p_pass_yards,
            p_pass_tds, p_ints, p_rush_attempts, p_rush_yards, p_rush_tds, p_receptions, p_receiving_yards, p_receiving_tds, 
            pass_attempts, completions, pass_yards, pass_tds, ints, rush_attempts, rush_yards, rush_tds, receptions, receiving_yards, receiving_tds])

            columns = ['season', 'week', 'Owner', 'PlayerName', 'PlayerID', 'Position', 'STARTER','is_injured', 'injury_status', 'fantasy_points', 'p_fantasy_points','p_pass_attempts', 'p_completions', 'p_pass_yards',
            'p_pass_tds', 'p_ints', 'p_rush_attempts', 'p_rush_yards', 'p_rush_tds', 'p_receptions', 'p_receiving_yards', 'p_receiving_tds', 
            'pass_attempts', 'completions', 'pass_yards', 'pass_tds', 'ints', 'rush_attempts', 'rush_yards', 'rush_tds', 'receptions', 'receiving_yards', 'receiving_tds']
            
        df = pd.DataFrame(output, columns=columns)
            
    return df

#player_df = get_weekly_player_stats(930603, 2022, 1)

In [321]:
def get_season_player_stats(leagueID, season, reg_season_length):

    for i in range(1, reg_season_length):
        df = pd.concat([get_weekly_player_stats(leagueID, season, i)])

    return df

In [322]:
get_season_player_stats(930603, 2022, 5 )

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


IndexError: list index out of range

In [ ]:
''' 
Function description...
'''

def heady_roster_moves(df):

    w = df['week'][0]
    s = df['season'][0]
    for name in OWNERS.items():

        team_name = name[1]
        team = df[df['Owner'] == team_name]
        healthy = team[team['injury_status'] == 'ACTIVE'] 
        starters = team[team['STARTER'] == 'TRUE']
        bench = team[team['STARTER'] == 'FALSE']

        for _, starter in starters.iterrows():

            s_name = starter['PlayerName']
            s_pos = starter['Position']
            flt_s_proj = starter['p_fantasy_points']
            int_s_proj = int(round(starter['p_fantasy_points'], 0))
            int_s_actual = int(round(starter['fantasy_points'], 0))
            flt_s_actual = starter['fantasy_points']
            
            for _, bencher in bench[bench['Position'] == s_pos].iterrows():

                b_name = bencher['PlayerName']
                b_pos = bencher['Position']
                flt_b_proj = bencher['p_fantasy_points']
                int_b_proj = int(round(bencher['p_fantasy_points'], 0))
                int_b_actual = int(round(bencher['fantasy_points'], 0))
                flt_b_actual = bencher['fantasy_points']

                if (int_b_proj > int_s_proj) and (int_s_actual > int_b_actual) :
                    print(f'In Week {w} of the {s} season, at {s_pos}, Team {team_name} started {s_name} (projected: {flt_s_proj} pts) over {b_name} (projected: {flt_b_proj}) who outscored {b_name} by {round(flt_s_actual - flt_b_actual, 1)} pts')